In [ ]:
import viennaps as ps # ViennaPS 2D 

### [MultiParticleProcess](https://viennatools.github.io/ViennaPS/models/prebuilt/multiParticle.html) – Modeling Multiple Particle Species

In [ ]:
# -------------------------------
# Helper function: Create domain with a simple masked trench
# -------------------------------
def createTrenchMask():
    extent = 30
    gridDelta = 0.3

    # Create a 2D simulation domain with specified extent and resolution
    domain = ps.Domain(xExtent=extent, gridDelta=gridDelta)

    # Add a rectangular trench with a flat bottom (depth=0) and mask layer on top
    ps.MakeTrench(domain, trenchWidth=10.0, trenchDepth=0.0, maskHeight=2.0).apply()

    return domain

### Step 1: Etching with a Single Neutral Particle

In [ ]:
# Initialize domain
domain = createTrenchMask()
domain.saveVolumeMesh("multiParticleEtching_1")

In [ ]:
# Create multi-particle process model
model = ps.MultiParticleProcess()

# Add a neutral particle source (isotropic direction, 20% sticking probability)
model.addNeutralParticle(stickingProbability=0.2)

# --- Parameter: control how strongly the neutral particle etches
neutralRate = 1.0

# Rate function using just the neutral particle
def rateFunction(fluxes, material):
    if material == ps.Material.Mask:
        return 0  # No etching of mask

    return -neutralRate * fluxes[0]

model.setRateFunction(rateFunction)

In [ ]:
processDuration = 5.0  # arbitrary time units
ps.Process(domain, model, processDuration).apply()
domain.saveVolumeMesh("multiParticleEtching_2")

### Step 2: Etching with Neutral + Ion Particles

In [ ]:
# Reset domain
domain = createTrenchMask()

In [ ]:
# Create new model with both particle types
model = ps.MultiParticleProcess()
model.addNeutralParticle(stickingProbability=0.2)
model.addIonParticle(sourcePower=500.0, thetaRMin=60.0, thetaRMax=90.0)

# --- Parameters: user-adjustable weights for particle contributions
neutralWeight    = 1.0   # influence of neutral particle
ionWeight        = 1.0   # influence of ion particle
maskEtchFactor   = 0.1   # how strongly ions etch the mask (0 = none, 1 = full)

# Updated rate function combining both particle fluxes
def rateFunction(fluxes, material):
    neutralFlux = fluxes[0]
    ionFlux = fluxes[1]

    if material == ps.Material.Mask:
        return -maskEtchFactor * ionWeight * ionFlux

    return -(neutralWeight * neutralFlux + ionWeight * ionFlux)

model.setRateFunction(rateFunction)

In [ ]:
processDuration = 5.0  # arbitrary time units
ps.Process(domain, model, processDuration).apply()
domain.saveVolumeMesh("multiParticleEtching_3")